# **PROJECT-3. EDA + Feature Engineering. Соревнование на Kaggle**

In [118]:
import pandas as pd 
import numpy as np
import re
import seaborn as sns
import geopy.distance # для работы с координатами
from matplotlib import pyplot as plt
from IPython.display import display
import logging
from comet_ml import Experiment


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки для обучения модели:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

ОПИСАНИЕ ДАННЫХ

1. **hotel_address** — адрес отеля;
2. **review_date** — дата, когда рецензент разместил соответствующий отзыв;
3. **average_score** — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
4. **hotel_name** — название отеля;
5. **reviewer_nationality** — страна рецензента;
6. **negative_review** — отрицательный отзыв, который рецензент дал отелю;
7. **review_total_negative_word_counts** — общее количество слов в отрицательном отзыв;
8. **positive_review** — положительный отзыв, который рецензент дал отелю;
9. **review_total_positive_word_counts** — общее количество слов в положительном отзыве.
10. **reviewer_score** — оценка, которую рецензент поставил отелю на основе своего опыта;
11. **total_number_of_reviews_reviewer_has_given** — количество отзывов, которые рецензенты дали в прошлом;
12. **total_number_of_reviews** — общее количество действительных отзывов об отеле;
13. **tags** — теги, которые рецензент дал отелю;
14. **days_since_review** — количество дней между датой проверки и датой очистки;
15. **additional_number_of_scoring** — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
16. **lat** — географическая широта отеля;
17. **lng** — географическая долгота отеля.

In [45]:
hotels = pd.read_csv('data/hotels.csv')
display(hotels.head(3))
display(hotels.info())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

None

***
### **Создание и преобразвание признаков**

Признак 1: **hotel_address**

- возможно получить страну
- возможно получить город

In [94]:
# получим страну и город из адреса отеля

# составим список всех стран из признака национальности автора обзора
country_set = set(hotels['reviewer_nationality'].str.strip())
country_set.remove('')
display(len(country_set)) # - всего 224 страны, этого должно хватить для вычленения страны из признака адреса отеля

# Функция для поиска страны с конца строки в нашем датафрейме
def get_countries(strs, countries):
    countries_series = []
    for string in strs:
        for country in countries:
            if country in ' '.join(string.split()[-2:]):
                countries_series.append(country)
    countries_series = pd.Series(countries_series)
    return countries_series

country_of_hotel = get_countries(hotels['hotel_address'],country_set)
hotels['country_of_hotel'] = country_of_hotel
display(country_of_hotel.value_counts())

224

United Kingdom    196774
Spain              45132
France             44830
Netherlands        43006
Austria            29178
Italy              27883
dtype: int64

In [116]:
# функция для поиска города отеля
def get_cities(strs,country):
    cities_series = []
    for string in range(0,strs.shape[0]):
        if country.iloc[string]=='United Kingdom':
            cities_series.append(''.join(strs.iloc[string].split()[-5:-2][0]))
        else:
            cities_series.append(''.join(strs.iloc[string].split()[-5:][3]))
    cities_series = pd.Series(cities_series)
    return cities_series

city_of_hotel = get_cities(hotels['hotel_address'],hotels['country_of_hotel'])
hotels['city_of_hotel'] = city_of_hotel
display(city_of_hotel.value_counts())

London       196774
Barcelona     45132
Paris         44830
Amsterdam     43006
Vienna        29178
Milan         27883
dtype: int64

### **Признак 2-3: latitude-longitude**

- можем вычислить примерное расстояние до центра города от отеля

In [121]:
hotels['city_of_hotel'][5]='Milan'

C:\Users\Роман\AppData\Local\Temp\ipykernel_16080\2989749930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotels['city_of_hotel'][5]='Milan'


In [125]:
# создадим признаки координат центра города (инфа взята с сайта https://www.gps-coordinates.net/)
latitude_by_city={'London':51.5073219,'Barcelona':41.3828939,'Paris':48.8588897,'Amsterdam':52.3727598,'Vienna':48.2083537,'Milan':45.4641943}
longitude_by_city={'London':-0.1276474,'Barcelona':2.1774322,'Paris':2.320041,'Amsterdam':4.8936041,'Vienna':16.3725042,'Milan':9.1896346}


latitude_by_city_list=[]
for i in range(0,hotels.shape[0]):
    for key, value in latitude_by_city.items():
        if key==hotels['city_of_hotel'][i]:
            latitude_by_city_list.append(value)
hotels['latitude_by_city']=latitude_by_city_list

longitude_by_city_list=[]
for i in range(0,hotels.shape[0]):
    for key, value in longitude_by_city.items():
        if key==hotels['city_of_hotel'][i]:
            longitude_by_city_list.append(value)
hotels['longitude_by_city']=longitude_by_city_list

display(hotels['longitude_by_city'])

# вычислим расстояние от отеля до центра города
#def get_center_dist(lat_1,long_1,lat_2,long_2):



0         -0.127647
1         -0.127647
2          2.320041
3          2.320041
4          4.893604
            ...    
386798    -0.127647
386799    16.372504
386800    -0.127647
386801    -0.127647
386802    -0.127647
Name: longitude_by_city, Length: 386803, dtype: float64